# trying to analyse the breats samples

In [2]:
import os
import gc
import pickle
import anndata
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.stats import spearmanr, pearsonr
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import spatialdm as sdm

In [3]:
path = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/breast_B1_1.h5ad"
chuvio_B1_1 = sc.read_h5ad(path)

In [5]:
chuvio_B1_1.obs

,orig.ident,nCount_originalexp,nFeature_originalexp,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,...,subsets_Mito_detected,subsets_Mito_percent,total,libsize_drop,sizeFactor,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters,ident
B1_1_aaaachep-1,B1,176.0,80,B1_1_aaaachep-1,1671.557617,2203.856201,176,0,0,0,...,0,0.0,176.0,False,1.117938,149.0,80,0,0,0
B1_1_aaaadpbi-1,B1,241.0,66,B1_1_aaaadpbi-1,1653.334229,2207.638916,241,0,0,0,...,0,0.0,241.0,False,1.530812,151.0,62,3,3,3
B1_1_aaaaepld-1,B1,125.0,53,B1_1_aaaaepld-1,1663.231079,2214.756104,125,0,0,0,...,0,0.0,125.0,False,0.793990,128.0,53,3,3,3
B1_1_aaabbgmk-1,B1,102.0,35,B1_1_aaabbgmk-1,1645.274536,2217.458984,102,0,0,0,...,0,0.0,102.0,False,0.647896,126.0,35,3,3,3
B1_1_aaabpgij-1,B1,110.0,55,B1_1_aaabpgij-1,1649.982544,2227.356445,110,0,0,0,...,0,0.0,110.0,False,0.698711,119.0,55,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B1_1_oiijncjg-1,B1,182.0,53,B1_1_oiijncjg-1,5991.727051,1417.232788,182,0,0,0,...,0,0.0,182.0,False,1.156049,149.0,53,3,3,3
B1_1_oiikcmdg-1,B1,144.0,56,B1_1_oiikcmdg-1,6009.589355,1417.129761,145,0,0,0,...,0,0.0,145.0,False,0.914676,142.0,56,3,3,3
B1_1_oiikehml-1,B1,46.0,26,B1_1_oiikehml-1,6042.314941,1416.647461,46,0,0,0,...,0,0.0,46.0,False,0.292188,102.0,31,15,15,15
B1_1_oiikpjgm-1,B1,22.0,17,B1_1_oiikpjgm-1,5984.858398,1380.782715,22,0,0,0,...,0,0.0,22.0,False,0.139742,87.0,23,5,5,5


In [7]:
path = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/breast_B1_1.h5ad"
chuvio_B1_1 = sc.read_h5ad(path)

chuvio_B1_1.obsm['spatial'] = chuvio_B1_1.obs[['x_centroid', 'y_centroid']].to_numpy()

chuvio_B1_1.raw = chuvio_B1_1

#running the method
sdm.weight_matrix(chuvio_B1_1, l=120, cutoff=0.2, single_cell=False) # weight_matrix by rbf kernel
sdm.extract_lr(chuvio_B1_1, 'human', min_cell=1)      # find overlapping LRs from CellChatDB
sdm.spatialdm_global(chuvio_B1_1, 100, specified_ind=None, method='both', nproc=1)     # global Moran selection (orig 1000 set to 100)
sdm.sig_pairs(chuvio_B1_1, method='permutation', fdr=True, threshold=0.1)     # select significant pairs
sdm.spatialdm_local(chuvio_B1_1, n_perm=100, method='both', specified_ind=None, nproc=1)     # local spot selection
sdm.sig_spots(chuvio_B1_1, method='permutation', fdr=False, threshold=0.1)     # significant local spots

chuvio_B1_1_sDM = chuvio_B1_1


100%|██████████| 100/100 [00:02<00:00, 48.63it/s]


In [8]:
#getting the top genes
chuvio_B1_1_sDM_top_G = chuvio_B1_1_sDM.uns['global_res'].sort_values(by='fdr')

chuvio_B1_1_sDM_top_G.to_csv('chuvio_B1_1_sDM_top_G.csv', index=False)


In [11]:
import os
import gc
import pickle
import anndata
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.stats import spearmanr, pearsonr
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import spatialdm as sdm

file = "breast_B1_2"
file_name = file

path = f"/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/{file}.h5ad"
file = sc.read_h5ad(path)

file.obsm['spatial'] = file.obs[['x_centroid', 'y_centroid']].to_numpy()

file.raw = file

#running the method
sdm.weight_matrix(file, l=120, cutoff=0.2, single_cell=False) # weight_matrix by rbf kernel
sdm.extract_lr(file, 'human', min_cell=1)      # find overlapping LRs from CellChatDB
sdm.spatialdm_global(file, 100, specified_ind=None, method='both', nproc=1)     # global Moran selection (orig 1000 set to 100)
sdm.sig_pairs(file, method='permutation', fdr=True, threshold=0.1)     # select significant pairs
sdm.spatialdm_local(file, n_perm=100, method='both', specified_ind=None, nproc=1)     # local spot selection
sdm.sig_spots(file, method='permutation', fdr=False, threshold=0.1)     # significant local spots

file = file.uns['global_res'].sort_values(by='fdr')
file.to_csv(f'{file_name}_sDM_top_G.csv', index=False)


100%|██████████| 100/100 [00:02<00:00, 42.03it/s]
